In [ ]:
import numpy as np
import pandas as pd

import astropy.units as u

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
%matplotlib inline
import seaborn as sns

import emcee

import scipy.io.idl

pal = sns.color_palette('colorblind', 6)

sns.set(color_codes=True)
sns.axes_style("white")
sns.set_style('whitegrid')

sns.palplot(pal)

# Read in MCMC Results

In [ ]:
labels = [r'$n_{e,0}$', r'$H_z$', r'$F_z$',"log prob", "log prior", r'$A_V$']
ndim = 3

## Mean Extinction Run

In [ ]:
filename = "Baade_fitting_ha_flare_r.h5"
reader = emcee.backends.HDFBackend(filename)

tau = reader.get_autocorr_time(tol = 0.)
burnin = int(2*np.max(tau))
thin = int(0.5*np.min(tau))
samples = reader.get_chain(discard=burnin, flat=True, thin=thin)
log_prob_samples = reader.get_log_prob(discard=burnin, flat=True, thin=thin)
log_prior_samples = reader.get_blobs(discard=burnin, flat=True, thin=thin)


all_samples = np.concatenate((
    samples, log_prob_samples[:, None], log_prior_samples[:, None]
), axis=1)

for i in range(ndim):
    mcmc = np.percentile(samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    if i == 0:
        n0_mcmc = [mcmc[1], q[0], q[1]]
    elif i == 1:
        hz_mcmc = [mcmc[1], q[0], q[1]]
    else:
        Fz_mcmc = [mcmc[1], q[0], q[1]]

In [ ]:
filename_hiAV = "Baade_fitting_ha_flare_r_hiAV.h5"
reader_hiAV = emcee.backends.HDFBackend(filename_hiAV)

tau = reader_hiAV.get_autocorr_time(tol = 0.)
burnin = int(2*np.max(tau))
thin = int(0.5*np.min(tau))
samples_hiAV = reader_hiAV.get_chain(discard=burnin, flat=True, thin=thin)
log_prob_samples_hiAV = reader_hiAV.get_log_prob(discard=burnin, flat=True, thin=thin)
log_prior_samples_hiAV = reader_hiAV.get_blobs(discard=burnin, flat=True, thin=thin)


all_samples_hiAV = np.concatenate((
    samples_hiAV, log_prob_samples_hiAV[:, None], log_prior_samples_hiAV[:, None]
), axis=1)

for i in range(ndim):
    mcmc_hiAV = np.percentile(samples_hiAV[:, i], [16, 50, 84])
    q = np.diff(mcmc_hiAV)
    if i == 0:
        n0_mcmc_hiAV = [mcmc_hiAV[1], q[0], q[1]]
    elif i == 1:
        hz_mcmc_hiAV = [mcmc_hiAV[1], q[0], q[1]]
    else:
        Fz_mcmc_hiAV = [mcmc_hiAV[1], q[0], q[1]]

In [ ]:
filename_loAV = "Baade_fitting_ha_flare_r_loAV.h5"
reader_loAV = emcee.backends.HDFBackend(filename_loAV)

tau = reader_loAV.get_autocorr_time(tol = 0.)
burnin = int(2*np.max(tau))
thin = int(0.5*np.min(tau))
samples_loAV = reader_loAV.get_chain(discard=burnin, flat=True, thin=thin)
log_prob_samples_loAV = reader_loAV.get_log_prob(discard=burnin, flat=True, thin=thin)
log_prior_samples_loAV = reader_loAV.get_blobs(discard=burnin, flat=True, thin=thin)


all_samples_loAV = np.concatenate((
    samples_loAV, log_prob_samples_loAV[:, None], log_prior_samples_loAV[:, None]
), axis=1)

for i in range(ndim):
    mcmc_loAV = np.percentile(samples_loAV[:, i], [16, 50, 84])
    q = np.diff(mcmc_loAV)
    if i == 0:
        n0_mcmc_loAV = [mcmc_loAV[1], q[0], q[1]]
    elif i == 1:
        hz_mcmc_loAV = [mcmc_loAV[1], q[0], q[1]]
    else:
        Fz_mcmc_loAV = [mcmc_loAV[1], q[0], q[1]]

In [ ]:
AV_type = np.array([r'$A_V$' for ell in log_prob_samples])

AV_type_hi = np.array([r'$A_V + \sigma_{Av}$' for ell in log_prob_samples_hiAV])

AV_type_lo = np.array([r'$A_V - \sigma_{Av}$' for ell in log_prob_samples_loAV])

AV_types_all = np.concatenate((AV_type_lo, AV_type, AV_type_hi))

master_samples = np.concatenate((
    all_samples_loAV, all_samples, all_samples_hiAV
), axis = 0)

master_data = pd.DataFrame({
    "{}".format(labels[0]):master_samples[:,0], 
    "{}".format(labels[1]):master_samples[:,1], 
    "{}".format(labels[2]):master_samples[:,2], 
    "{}".format(labels[3]):master_samples[:,3], 
    "{}".format(labels[4]):master_samples[:,4], 
    "{}".format(labels[5]):AV_types_all, 
})

In [ ]:
list_of_cmaps = [sns.light_palette(pal[0], as_cmap = True), 
                 sns.light_palette(pal[1], as_cmap = True), 
                 sns.light_palette(pal[2], as_cmap = True)]

g = sns.PairGrid(master_data, 
                 palette = pal, 
                 vars = [r'$n_{e,0}$', r'$H_z$', r'$F_z$'], 
                 hue = r'$A_V$', 
                 hue_kws={"cmap":list_of_cmaps})

labels = [r'$n_{e,0}$', r'$H_z$', r'$F_z$']


# g = g.map_diag(plt.hist, histtype="step", linewidth=3, bins = 30, alpha = .7)
g = g.map_diag(sns.distplot)
g = g.map_lower(sns.kdeplot, shade_lowest = False, shade = True, alpha = .5)
g = g.map_upper(plt.scatter, alpha = 0.05, s = 1)


for i in range(3):
    #ne
    ylim = g.axes[i,0].get_ylim()
    g.axes[i,0].plot([n0_mcmc[0], n0_mcmc[0]],
                     [-1, 1600],
                     '-', 
                     c = pal[1], 
                     lw = 2)
    g.axes[i,0].plot([n0_mcmc[0]-n0_mcmc[1], 
                      n0_mcmc[0]-n0_mcmc[1]],
                     [-1, 1600],
                     '--', 
                     c = pal[1], 
                     lw = 1)
    g.axes[i,0].plot([n0_mcmc[0]+n0_mcmc[2], 
                      n0_mcmc[0]+n0_mcmc[2]],
                     [-1, 1600],
                     '--', 
                     c = pal[1], 
                     lw = 1)
    
    # AV Errors
    g.axes[i,0].plot([n0_mcmc_hiAV[0], n0_mcmc_hiAV[0]],
                     [-1, 1600],
                     ':', 
                     c = pal[2], 
                     lw = 2)

    g.axes[i,0].plot([n0_mcmc_loAV[0], n0_mcmc_loAV[0]],
                     [-1, 1600],
                     ':', 
                     c = pal[0], 
                     lw = 2)
    g.axes[i,0].plot([n0_mcmc_hiAV[0]+n0_mcmc_hiAV[2], n0_mcmc_hiAV[0]+n0_mcmc_hiAV[2]],
                     [-1, 1600],
                     '--', 
                     c = pal[2], 
                     lw = 2)

    g.axes[i,0].plot([n0_mcmc_loAV[0]-n0_mcmc_hiAV[1], n0_mcmc_loAV[0]-n0_mcmc_hiAV[1]],
                     [-1, 1600],
                     '--', 
                     c = pal[0], 
                     lw = 2)

    
    
    g.axes[i,0].set_ylim(ylim)
    #Hz
    ylim = g.axes[i,1].get_ylim()
    g.axes[i,1].plot([hz_mcmc[0], 
                      hz_mcmc[0]],
                     [-1, 1600],
                     '-', 
                     c = pal[1], 
                     lw = 2)
    g.axes[i,1].plot([hz_mcmc[0]-hz_mcmc[1], 
                      hz_mcmc[0]-hz_mcmc[1]],
                     [-1, 1600],
                     '--', 
                     c = pal[1], 
                     lw = 1)
    g.axes[i,1].plot([hz_mcmc[0]+hz_mcmc[2], 
                      hz_mcmc[0]+hz_mcmc[2]],
                     [-1, 1600],
                     '--', 
                     c = pal[1], 
                     lw = 1)
    
    # AV Errors
    g.axes[i,1].plot([hz_mcmc_hiAV[0], hz_mcmc_hiAV[0]],
                     [-1, 1600],
                     ':', 
                     c = pal[2], 
                     lw = 2)

    g.axes[i,1].plot([hz_mcmc_loAV[0], hz_mcmc_loAV[0]],
                     [-1, 1600],
                     ':', 
                     c = pal[0], 
                     lw = 2)
    
    
    g.axes[i,1].set_ylim(ylim)
    
    #Fz
    ylim = g.axes[i,2].get_ylim()
    g.axes[i,2].plot([Fz_mcmc[0], 
                      Fz_mcmc[0]],
                     [-1, 1600],
                     '-', 
                     c = pal[1], 
                     lw = 2)
    g.axes[i,2].plot([Fz_mcmc[0]-hz_mcmc[1], 
                      Fz_mcmc[0]-hz_mcmc[1]],
                     [-1, 1600],
                     '--', 
                     c = pal[1], 
                     lw = 1)
    g.axes[i,2].plot([Fz_mcmc[0]+hz_mcmc[2], 
                      Fz_mcmc[0]+hz_mcmc[2]],
                     [-1, 1600],
                     '--', 
                     c = pal[1], 
                     lw = 1)
    
    # AV Errors
    g.axes[i,2].plot([Fz_mcmc_hiAV[0], Fz_mcmc_hiAV[0]],
                     [-1, 1600],
                     ':', 
                     c = pal[2], 
                     lw = 2)

    g.axes[i,2].plot([Fz_mcmc_loAV[0], Fz_mcmc_loAV[0]],
                     [-1, 1600],
                     ':', 
                     c = pal[0], 
                     lw = 2)
    
    
    g.axes[i,2].set_ylim(ylim)
    
g.axes[0,0].set_xlabel(r'{0} $= {1:.2f}_{{-{2:.2f}}}^{{{3:.2f}}}$'.format(labels[0],
                                                                        n0_mcmc[0], 
#                                                                         n0_mcmc[1], 
#                                                                         n0_mcmc[2], 
                                                n0_mcmc[0] - (n0_mcmc_loAV[0] - n0_mcmc_loAV[1]), 
                                                (n0_mcmc_hiAV[0]+n0_mcmc_hiAV[2]) - n0_mcmc[0]), 
                       fontsize = 12)
g.axes[0,0].xaxis.set_label_position('top') 
g.axes[0,1].set_xlabel(r'{0} $= {1:.2f}_{{-{2:.2f}}}^{{{3:.2f}}}$'.format(labels[1],
                                                                        hz_mcmc[0], 
                                                                        hz_mcmc[1], 
                                                                        hz_mcmc[2]), 
                       fontsize = 12)
g.axes[0,1].xaxis.set_label_position('top') 
g.axes[0,2].set_xlabel(r'{0} $= {1:.2f}_{{-{2:.2f}}}^{{{3:.2f}}}$'.format(labels[2],
                                                                        Fz_mcmc[0], 
                                                                        Fz_mcmc[1], 
                                                                        Fz_mcmc[2]), 
                       fontsize = 12) 
g.axes[0,2].xaxis.set_label_position('top') 



xlim = g.axes[0,1].get_xlim()
g.axes[0,1].plot([0,5], 
                 [n0_mcmc[0], n0_mcmc[0]], 
                 '-', c = pal[1], lw = 2)
g.axes[0,1].plot([0,5], 
                 [n0_mcmc[0]-n0_mcmc[1], n0_mcmc[0]-n0_mcmc[1]], 
                 '--', c = pal[1], lw = 1)
g.axes[0,1].plot([0,5], 
                 [n0_mcmc[0]+n0_mcmc[2], n0_mcmc[0]+n0_mcmc[2]], 
                 '--', c = pal[1], lw = 1)

g.axes[0,1].plot([0,5], 
                 [n0_mcmc_hiAV[0], n0_mcmc_hiAV[0]], 
                 ':', c = pal[2], lw = 2)
g.axes[0,1].plot([0,5], 
                 [n0_mcmc_loAV[0], n0_mcmc_loAV[0]], 
                 ':', c = pal[0], lw = 2)
g.axes[0,1].plot([0,5], 
                 [n0_mcmc_hiAV[0]+n0_mcmc_hiAV[2], n0_mcmc_hiAV[0]+n0_mcmc_hiAV[2]], 
                 '--', c = pal[2], lw = 2)
g.axes[0,1].plot([0,5], 
                 [n0_mcmc_loAV[0]-n0_mcmc_loAV[1], n0_mcmc_loAV[0]-n0_mcmc_loAV[1]], 
                 '--', c = pal[0], lw = 2)


g.axes[0,1].set_xlim(xlim)
xlim = g.axes[0,2].get_xlim()
g.axes[0,2].plot([0,5], 
                 [n0_mcmc[0], n0_mcmc[0]], 
                 '-', c = pal[1], lw = 2)
g.axes[0,2].plot([0,5], 
                 [n0_mcmc[0]-n0_mcmc[1], n0_mcmc[0]-n0_mcmc[1]], 
                 '--', c = pal[1], lw = 1)
g.axes[0,2].plot([0,5], 
                 [n0_mcmc[0]+n0_mcmc[2], n0_mcmc[0]+n0_mcmc[2]], 
                 '--', c = pal[1], lw = 1)

g.axes[0,2].plot([0,5], 
                 [n0_mcmc_hiAV[0], n0_mcmc_hiAV[0]], 
                 ':', c = pal[2], lw = 2)
g.axes[0,2].plot([0,5], 
                 [n0_mcmc_loAV[0], n0_mcmc_loAV[0]], 
                 ':', c = pal[0], lw = 2)
g.axes[0,2].plot([0,5], 
                 [n0_mcmc_hiAV[0]+n0_mcmc_hiAV[2], n0_mcmc_hiAV[0]+n0_mcmc_hiAV[2]], 
                 '--', c = pal[2], lw = 2)
g.axes[0,2].plot([0,5], 
                 [n0_mcmc_loAV[0]-n0_mcmc_loAV[1], n0_mcmc_loAV[0]-n0_mcmc_loAV[1]], 
                 '--', c = pal[0], lw = 2)
g.axes[0,2].set_xlim(xlim)


xlim = g.axes[1,0].get_xlim()
g.axes[1,0].plot([0,5], 
                 [hz_mcmc[0], hz_mcmc[0]], 
                 '-', c = pal[1], lw = 2)
g.axes[1,0].plot([0,5], 
                 [hz_mcmc[0]-hz_mcmc[1], hz_mcmc[0]-hz_mcmc[1]], 
                 '--', c = pal[1], lw = 1)
g.axes[1,0].plot([0,5], 
                 [hz_mcmc[0]+hz_mcmc[2], hz_mcmc[0]+hz_mcmc[2]], 
                 '--', c = pal[1], lw = 1)

g.axes[1,0].plot([0,5], 
                 [hz_mcmc_hiAV[0], hz_mcmc_hiAV[0]], 
                 ':', c = pal[2], lw = 2)
g.axes[1,0].plot([0,5], 
                 [hz_mcmc_loAV[0], hz_mcmc_loAV[0]], 
                 ':', c = pal[0], lw = 2)

g.axes[1,0].set_xlim(xlim)
xlim = g.axes[1,2].get_xlim()
g.axes[1,2].plot([0,5], 
                 [hz_mcmc[0], hz_mcmc[0]], 
                 '-', c = pal[1], lw = 2)
g.axes[1,2].plot([0,5], 
                 [hz_mcmc[0]-hz_mcmc[1], hz_mcmc[0]-hz_mcmc[1]], 
                 '--', c = pal[1], lw = 1)
g.axes[1,2].plot([0,5], 
                 [hz_mcmc[0]+hz_mcmc[2], hz_mcmc[0]+hz_mcmc[2]], 
                 '--', c = pal[1], lw = 1)

g.axes[1,2].plot([0,5], 
                 [hz_mcmc_hiAV[0], hz_mcmc_hiAV[0]], 
                 ':', c = pal[2], lw = 2)
g.axes[1,2].plot([0,5], 
                 [hz_mcmc_loAV[0], hz_mcmc_loAV[0]], 
                 ':', c = pal[0], lw = 2)

g.axes[1,2].set_xlim(xlim)


xlim = g.axes[2,0].get_xlim()
g.axes[2,0].plot([0,5], 
                 [Fz_mcmc[0], Fz_mcmc[0]], 
                 '-', c = pal[1], lw = 2)
g.axes[2,0].plot([0,5], 
                 [Fz_mcmc[0]-Fz_mcmc[1], Fz_mcmc[0]-Fz_mcmc[1]], 
                 '--', c = pal[1], lw = 1)
g.axes[2,0].plot([0,5], 
                 [Fz_mcmc[0]+Fz_mcmc[2], Fz_mcmc[0]+Fz_mcmc[2]], 
                 '--', c = pal[1], lw = 1)

g.axes[2,0].plot([0,5], 
                 [Fz_mcmc_hiAV[0], Fz_mcmc_hiAV[0]], 
                 ':', c = pal[2], lw = 2)
g.axes[2,0].plot([0,5], 
                 [Fz_mcmc_loAV[0], Fz_mcmc_loAV[0]], 
                 ':', c = pal[0], lw = 2)

g.axes[2,0].set_xlim(xlim)
xlim = g.axes[2,1].get_xlim()
g.axes[2,1].plot([0,5], 
                 [Fz_mcmc[0], Fz_mcmc[0]], 
                 '-', c = pal[1], lw = 2)
g.axes[2,1].plot([0,5], 
                 [Fz_mcmc[0]-Fz_mcmc[1], Fz_mcmc[0]-Fz_mcmc[1]], 
                 '--', c = pal[1], lw = 1)
g.axes[2,1].plot([0,5], 
                 [Fz_mcmc[0]+Fz_mcmc[2], Fz_mcmc[0]+Fz_mcmc[2]], 
                 '--', c = pal[1], lw = 1)

g.axes[2,1].plot([0,5], 
                 [Fz_mcmc_hiAV[0], Fz_mcmc_hiAV[0]], 
                 ':', c = pal[2], lw = 2)
g.axes[2,1].plot([0,5], 
                 [Fz_mcmc_loAV[0], Fz_mcmc_loAV[0]], 
                 ':', c = pal[0], lw = 2)

g.axes[2,1].set_xlim(xlim)

g.axes[0,2].set_ylabel(r'$[cm^{-3}]$', fontsize = 12)
g.axes[0,2].yaxis.set_label_position('right')

g.axes[1,2].set_ylabel(r'$[kpc]$', fontsize = 12)
g.axes[1,2].yaxis.set_label_position('right')

g.axes[2,2].set_ylabel(r'$[\#]$', fontsize = 12)
g.axes[2,2].yaxis.set_label_position('right')


handles = g._legend_data.values()
labels = g._legend_data.keys()

leg = plt.legend(handles=handles, labels=labels, loc='upper right', 
                 ncol=1, markerscale = 5, fontsize = 12)

for lh in leg.legendHandles: 
    lh.set_alpha(1)

    
    